# Multi-circle selection over plot

In [5]:
%matplotlib qt
import matplotlib.pyplot as plt
import numpy as np

from matplotlib.widgets import EllipseSelector, RectangleSelector
import matplotlib.patches as patches
from matplotlib.backend_bases import MouseButton

In [101]:
class Circle:
    """Class to handle circle creation and properties
    """    
    def __init__(self, ax, center, radius, **kwargs):
        self.ax = ax
        self.center = center
        self.radius = radius
        self.circ = patches.Circle(center, radius, **kwargs)
        self.center_crosshair = self.ax.scatter(*self.center, c='r', marker='+')
        self.ax.add_patch(self.circ)
        self.ax.figure.canvas.draw_idle()
    
    def contains_event(self, event):
        return self.circ.contains(event)[0]
    
    def remove(self):
        self.circ.remove()
        self.center_crosshair.remove()
        self.ax.figure.canvas.draw_idle()
    
    def set_offsets(self, center):
        self.center = center
        self.circ.center = center
        self.center_crosshair.set_offsets(center)
        self.ax.figure.canvas.draw_idle()
    
    def set_radius(self, radius):
        self.radius = radius
        self.circ.radius = radius
        self.ax.figure.canvas.draw_idle()

    def set_edge_style(self, linestyle, linewidth):
        self.circ.set_linestyle(linestyle)
        self.circ.set_linewidth(linewidth)
        self.ax.figure.canvas.draw_idle()
    
class CircleSelector:
    """Class to handle multi circle selections

    Returns
    -------
    _type_
        _description_
    """    
    SCROLL_STEP = 0.05
    MIN_RADIUS = 0.05
    def __init__(self, ax):
        fig.canvas.mpl_connect('button_press_event', self.on_press)
        fig.canvas.mpl_connect('scroll_event', self.on_scroll)
        fig.canvas.mpl_connect('button_release_event', self.on_release)
        fig.canvas.mpl_connect('motion_notify_event', self.on_move)
        self.axes = ax
        self.circles = []  # List to store all circles
        self.active_circle = None
        self.pressevent = None

    def get_active_circle(self, event):
        """Get the circle that contains the event, if any

        Parameters
        ----------
        event : _type_
            _description_

        Returns
        -------
        _type_
            _description_
        """        
        for circle in self.circles:
            if circle.contains_event(event):
                return circle
        return None

    def on_press(self, event):
        """Create a circle with middle click or move an existing circle with left click

        Parameters
        ----------
        event : _type_
            _description_
        """        
        if event.inaxes != self.axes:
            return
        self.clear_all_circle_selections()
        if event.button == MouseButton.MIDDLE:
            self.circles.append(Circle(self.axes, (event.xdata, event.ydata), 0.25, alpha=0.1, fc='yellow', ec='red'))
            self.active_circle = self.get_active_circle(event)
        elif event.button == MouseButton.LEFT:
            self.active_circle = self.get_active_circle(event)
            if self.active_circle is not None:
                self.pressevent = event
                # Store the initial press coordinates and the circle's initial center
                self.initial_press_xdata = event.xdata
                self.initial_press_ydata = event.ydata
                self.initial_circle_center = self.active_circle.center
                # Add contour to the circle
                self.active_circle.set_edge_style('dashed', 1)
                self.axes.figure.canvas.draw_idle()

    def clear_all_circle_selections(self):
        """Clear all circle selections"""   
        for circle in self.circles:
            circle.set_edge_style('solid', 0)
        self.axes.figure.canvas.draw_idle()

    def on_scroll(self, event):
        """Increase or decrease circle radius with scroll wheel

        Parameters
        ----------
        event : _type_
            _description_
        """
        if event.inaxes != self.axes:
            return
        if self.active_circle is None:
            return
        if self.active_circle.contains_event(event):
            increment = self.SCROLL_STEP if event.button == 'up' else -self.SCROLL_STEP
            self.active_circle.set_radius(self.active_circle.radius + increment)
            if self.active_circle.radius < self.MIN_RADIUS:
                self.active_circle.remove()
                self.circles.remove(self.active_circle)
                self.active_circle = None

    def on_move(self, event):
        """Move circle with left click

        Parameters
        ----------
        event : _type_
            _description_
        """
        if not hasattr(self, 'pressevent'):
            return
        # If there is no press event (circle was not clicked) or the mouse is not in the press event axes, return
        if self.pressevent is None or event.inaxes != self.pressevent.inaxes:
            return
        # If there is no circle drawn or the mouse is not in the circle, return
        if self.active_circle is None:
            return
        dx = event.xdata - self.initial_press_xdata
        dy = event.ydata - self.initial_press_ydata
        new_center = (self.initial_circle_center[0] + dx, self.initial_circle_center[1] + dy)
        self.active_circle.set_offsets(new_center)


    def on_release(self, event):
        """Reset press event

        Parameters
        ----------
        event : _type_
            _description_
        """        
        if self.pressevent is not None:
            self.pressevent = None
            
fig = plt.figure(layout='constrained')
axs = fig.subplots(1)

N = 1000 
x = np.linspace(0, 1, N)
y = np.random.gamma(2, size=N) + x

axs.scatter(x, y, s=0.1)

axs.axis([0, 1, 0, 1])
handler = CircleSelector(axs)
plt.show()